In [1]:
#Importing all the necessary libraries
import cv2
import numpy as np
from sklearn.metrics import pairwise

DETECTING FINGERS :

1.Defining an ROI where fingers will be detected.

2.Running background analysis(Getting average background value) to study the background so that elements in the backgrounds would not be mistook as fingers.

3.Finding the largest contour in the ROI assuming it to be the hand.

4.Obtaining the topmost , rightend and leftend (extreme points) and finding the co-ordinates of the center of the hand by using convexHull() method.

5.Finding the euclidean distances between the center and the extreme points and storing the maximum one.

5.Creating a circular ROI with radius being 90% of the maximum euclidean distance and masking it on the thresholded version.

6.All the external contours are then detected in the imaginary circular ROI and contours which lie in the limit_points(circumference of the circular ROI) and out_of_wrist (Height greater than  wrist points) are detected as fingers and no.of finger counters are counted hence giving the no.of fingers.

In [2]:
#Creating required global variables

bg = None
accumulated_wt = 1.5 #Default values

#Defining the position of the ROI
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
#Function to get the background and get the accumulated weights

def get_bg_wt(frame,accumulated_wt):
    global bg
    
    if bg is None:
        bg = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame,bg,accumulated_wt)

In [4]:
#Segmenting hand based on ROI

def segment_hand(frame,threshold = 25):
    
    diff = cv2.absdiff(bg.astype('uint8'),frame)

    ret,thresh = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    
    contours,_ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     _,contours = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours,key = cv2.contourArea)
    return(thresh,hand_segment)

In [5]:
def count_fingers(thresh,hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    
    conv_hull = cv2.convexHull(hand_segment)
    
    top = conv_hull(conv_hull[:,:,1].argmin()[0])
    bottom = conv_hull(conv_hull[:,:,1].argmax()[0])
    left = conv_hull(conv_hull[:,:,0].argmin()[0])
    right = conv_hull(conv_hull[:,:,0].argmax()[0])
    
    cX  = (left[0]+right[0])//2
    cY  = (top[1]+bottom[1])//2
    
    distance = pairwise.euclidean_distances([cX,cY],Y = [left,right,top,bottom])[0]
    max_dist = distance.max()
    radius = int(0.8*max_dist)
    circumference  = (2*np.pi*radius)
    
    circular_roi = np.zeros(thresh[:2],dtype = 'uint8')
    cv2.cirlce(circular_roi,(cX,cY),radius,255,10)
    circular_roi = cv2.bitwise_and(thresh,thresh,mask = circular_roi)
    
    fing_cont,_ = cv2.findContours(circular_roi,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    count = 0
    
    for cont in fing_cont:
        (x,y,w,h) = cv2.boundingRect(cont)
        
        out_of_wrist = (cY+(cY*0.25)) > (y+h)
        
        limit_points = ((0.25*circumference) > cnt.shape[0])
        
        if out_of_wrist and limit_points :
            count +=1
            
    return count
    
    
        

In [6]:
cap = cv2.VideoCapture(0)

num_frames = 0

while True:
        ret,frame = cap.read()
        frame_copy = frame.copy()
        
        roi = frame[roi_top:roi_bottom,roi_right:roi_left]
        gray = cv2.cvtColor(frame_copy,cv2.COLOR_BGR2RGB)
        gray = cv2.GaussianBlur(gray,(7,7),0)
        
        if num_frames <  60:
            get_bg_wt(gray,accumulated_wt)
            
            if num_frames<=59:
                cv2.putText(frame_copy,"Work In Progress",(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),4)
        else:
            hand = segment_hand(gray)
            if hand is not None:
                threshold,hand_segment = hand
                cv2.drawContours(frame_copy,[hand_segment],-1,(255,0,0),5)
                
                fingers_count = count_fingers(threshold,hand_segment)
                cv2.putText(frame_copy,str(fingers_count),(70,50),cv2.FONT_HERSHEY_SIMPLEX,(255,255,255),3)
                
                cv2.imshow('GrayScale ROI',threshold)
                
        cv2.rectangle(frame_copy,(roi_right,roi_top),(roi_left,roi_bottom),(0,0,255),3)
        num_frames += 1
        cv2.imshow('Finger_count',frame_copy)
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
cap.release()
cv2.destroyAllWindows()
                
        

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-k8sx3e60\opencv\modules\imgproc\src\contours.cpp:197: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'
